In [1]:
! pip install jupyterlab streamlit librosa scikit-learn sounddevice numpy scipy


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
! pip install sounddevice numpy scipy


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import sounddevice as sd
import numpy as np
import scipy.io.wavfile as wav
import os
import time

In [4]:
# --- Konfigurasi ---
JUMLAH_SAMPEL = 100      # 100x rekaman per kata
DURASI_REKAMAN = 1.5    # Durasi rekaman dalam detik (sesuaikan jika perlu)
SAMPLE_RATE = 22050     # Standar untuk speech recognition
KATA_TARGET = ["buka", "tutup"]
FOLDER_DATA = "./dataset/data-hasil"
# --------------------

def rekam_audio(durasi, sr):
    """Merekam audio dari mikrofon default."""
    print("Merekam...")
    audio = sd.rec(int(durasi * sr), samplerate=sr, channels=1, dtype='float32')
    sd.wait()  # Menunggu rekaman selesai
    print("Rekaman selesai.")
    return audio

def simpan_audio(audio, filename, sr):
    """Menyimpan audio ke file WAV."""
    # Normalisasi audio untuk menghindari clipping (jika terlalu keras)
    audio = audio / np.max(np.abs(audio))
    # Konversi ke 16-bit integer (format WAV standar)
    audio_int16 = (audio * 32767).astype(np.int16)
    wav.write(filename, sr, audio_int16)
    print(f"File disimpan di: {filename}")

def main():
    print("--- Skrip Perekam Data Audio ---")
    print(f"Target: {JUMLAH_SAMPEL} sampel per kata.")
    print(f"Kata: {', '.join(KATA_TARGET)}")
    print("----------------------------------\n")

    # Membuat folder data utama jika belum ada
    if not os.path.exists(FOLDER_DATA):
        os.makedirs(FOLDER_DATA)

    for kata in KATA_TARGET:
        print(f"\n======= MULAI MEREKAM KATA: '{kata.upper()}' =======")
        
        # Membuat sub-folder untuk setiap kata
        folder_kata = os.path.join(FOLDER_DATA, kata)
        if not os.path.exists(folder_kata):
            os.makedirs(folder_kata)
        
        print(f"Data akan disimpan di folder: {folder_kata}")
        print("Tekan ENTER untuk memulai rekaman pertama...")
        input() # Menunggu kesiapan pengguna

        for i in range(JUMLAH_SAMPEL):
            print(f"\n--- Merekam sampel ke-{i+1}/{JUMLAH_SAMPEL} untuk '{kata.upper()}' ---")
            
            # Beri waktu pengguna untuk bersiap
            try:
                print("Bersiap dalam 3...")
                time.sleep(1)
                print("2...")
                time.sleep(1)
                print("1...")
                time.sleep(1)
                print("BICARA SEKARANG!")

                # 1. Rekam
                audio_rekaman = rekam_audio(DURASI_REKAMAN, SAMPLE_RATE)
                
                # 2. Simpan
                # Format nama file: buka_001.wav, buka_002.wav, ...
                nama_file = f"{kata}_{i+1:03d}.wav" 
                path_file_lengkap = os.path.join(folder_kata, nama_file)
                simpan_audio(audio_rekaman, path_file_lengkap, SAMPLE_RATE)

                # Jeda singkat antar rekaman
                time.sleep(0.5)

            except KeyboardInterrupt:
                print("\nProses dihentikan oleh pengguna.")
                return
            except Exception as e:
                print(f"Terjadi error: {e}")
                print("Mencoba lanjut ke sampel berikutnya...")
                continue
    
    print("\n\n======== PENGUMPULAN DATA SELESAI! ========")
    print(f"Semua data tersimpan di folder '{FOLDER_DATA}'.")

if __name__ == "__main__":
    main()

--- Skrip Perekam Data Audio ---
Target: 100 sampel per kata.
Kata: buka, tutup
----------------------------------


======= MULAI MEREKAM KATA: 'BUKA' =======
Data akan disimpan di folder: ./dataset/data-hasil\buka
Tekan ENTER untuk memulai rekaman pertama...


StdinNotImplementedError: raw_input was called, but this frontend does not support input requests.

In [ ]:
!pip install librosa pandas numpy tqdm

In [6]:
import librosa
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

# Tentukan path ke folder data Anda
PATH_DATA = "./dataset/data-hasil/"
LABELS = ["buka", "tutup"]
SAMPLE_RATE = 22050

# --- PENYESUAIAN ---
# Mengubah jumlah MFCC dari 13 menjadi 20
N_MFCC = 20 
# -------------------

# Siapkan list untuk menampung data
data_fitur = []
data_label = []

print(f"Memulai ekstraksi fitur MFCC (n_mfcc={N_MFCC})...")

# Looping untuk setiap label (buka, tutup)
for label in LABELS:
    folder_path = os.path.join(PATH_DATA, label)
    
    # Dapatkan semua nama file di dalam folder
    files = [f for f in os.listdir(folder_path) if f.endswith('.wav')]
    
    # Gunakan tqdm untuk melihat progress bar
    for file_name in tqdm(files, desc=f"Processing {label}"):
        try:
            file_path = os.path.join(folder_path, file_name)
            
            # 1. Muat file audio
            y, sr = librosa.load(file_path, sr=SAMPLE_RATE)
            
            # 2. Ekstrak fitur MFCC (dengan n_mfcc=20)
            mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=N_MFCC)
            
            # 3. Ambil rata-rata MFCC di semua frame
            mfccs_rata_rata = np.mean(mfccs.T, axis=0)
            
            # 4. Simpan fitur dan label
            data_fitur.append(mfccs_rata_rata)
            data_label.append(label)
            
        except Exception as e:
            print(f"Error memproses file {file_path}: {e}")

print("\nEkstraksi fitur selesai.")

# --- Konversi ke DataFrame Pandas ---

# Buat DataFrame dari list fitur
df = pd.DataFrame(data_fitur)

# Beri nama kolom (mfcc_0, mfcc_1, ..., mfcc_19)
df = df.add_prefix('mfcc_')

# Tambahkan kolom label
df['label'] = data_label

# --- Simpan ke CSV ---
NAMA_FILE_CSV = "./dataset/data-hasil/fitur_audio_20mfcc.csv"
df.to_csv(NAMA_FILE_CSV, index=False)

print(f"Data fitur berhasil disimpan ke {NAMA_FILE_CSV}")
print("\nContoh 5 data pertama:")
print(df.head())

Memulai ekstraksi fitur MFCC (n_mfcc=20)...


Processing tutup: 100%|██████████| 100/100 [00:01<00:00, 65.54it/s]


Ekstraksi fitur selesai.
Data fitur berhasil disimpan ke ./dataset/data-hasil/fitur_audio_20mfcc.csv

Contoh 5 data pertama:
       mfcc_0     mfcc_1     mfcc_2     mfcc_3     mfcc_4     mfcc_5  \
0 -391.825195  86.733055  21.756060  13.757183   9.030744  12.091555   
1 -219.554581  95.143524  23.050543  11.273750  12.111441   7.253877   
2 -231.499649  95.491196  21.786690  16.874832   9.731382  10.766232   
3 -239.903519  93.402954  20.391285  14.796158   7.593359   8.971307   
4 -239.522278  96.243011  13.798207  11.316676   9.502037   7.641552   

      mfcc_6    mfcc_7    mfcc_8    mfcc_9  ...   mfcc_11   mfcc_12   mfcc_13  \
0  12.048173  0.365709  1.737657  5.475190  ... -6.118983 -5.571948 -4.757148   
1  12.820862 -2.045698 -0.279506  7.276806  ... -7.047171 -9.191520 -6.422854   
2  14.763392  2.315336  0.482845  3.260504  ... -1.712604 -7.362328 -1.683394   
3  12.700639 -0.081058  0.001442  4.289254  ... -2.979377 -8.099751 -2.410700   
4  16.703917  0.591387 -1.314283  5.

In [ ]:
!pip install scikit-learn joblib

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import joblib

# --- 1. Muat Data ---
NAMA_FILE_CSV = "./dataset/data-hasil/fitur_audio_20mfcc.csv"
df = pd.read_csv(NAMA_FILE_CSV)

print("Data berhasil dimuat:")
print(df.head())

# --- 2. Pra-pemrosesan Data ---

# Pisahkan fitur (X) dan label (y)
X = df.drop('label', axis=1) # Ambil semua kolom kecuali 'label'
y = df['label']

# Ubah label "buka" dan "tutup" menjadi angka (misal: 0 dan 1)
# Ini WAJIB karena model hanya mengerti angka
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

# Mari kita lihat kelasnya (0 itu apa, 1 itu apa)
print(f"\nKelas label: {encoder.classes_}")
# Ini akan mencetak sesuatu seperti: ['buka' 'tutup']
# Artinya: 'buka' = 0, 'tutup' = 1

# --- 3. Bagi Data (Train & Test Split) ---
# Kita akan gunakan 80% data untuk melatih model, 20% untuk menguji
# stratify=y_encoded: Memastikan proporsi 'buka' dan 'tutup' seimbang di data latih dan uji
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, 
                                                    test_size=0.2, 
                                                    random_state=42, 
                                                    stratify=y_encoded)

print(f"\nBentuk data latih (X_train): {X_train.shape}")
print(f"Bentuk data tes (X_test):   {X_test.shape}")

# --- 4. Buat dan Latih Model ---
print("\nMemulai pelatihan model SVC...")

# Kita gunakan SVC (Support Vector Classifier)
# kernel='linear' biasanya bekerja baik untuk data teks/audio
model = SVC(kernel='linear')

# Latih model menggunakan data latih
model.fit(X_train, y_train)

print("Pelatihan model selesai.")

# --- 5. Evaluasi Model ---
print("\n--- Hasil Evaluasi Model ---")
# Gunakan model yang sudah dilatih untuk menebak data tes
y_pred = model.predict(X_test)

# Hitung akurasi
accuracy = accuracy_score(y_test, y_pred)
print(f"Akurasi Model: {accuracy * 100:.2f}%")

# Tampilkan laporan detail (precision, recall, f1-score)
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=encoder.classes_))

# Tampilkan Confusion Matrix
# Ini menunjukkan berapa banyak 'buka' yang ditebak benar 'buka',
# berapa 'buka' yang salah tebak jadi 'tutup', dst.
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# --- 6. Simpan Model ---
# Ini adalah langkah KRUSIAL untuk implementasi Streamlit
NAMA_FILE_MODEL = './dataset/data-hasil/model_audio.pkl'
NAMA_FILE_ENCODER = './dataset/data-hasil/label_encoder.pkl'

joblib.dump(model, NAMA_FILE_MODEL)
joblib.dump(encoder, NAMA_FILE_ENCODER) # Kita juga simpan encoder-nya!

print(f"\nModel berhasil disimpan ke: {NAMA_FILE_MODEL}")
print(f"Encoder berhasil disimpan ke: {NAMA_FILE_ENCODER}")

Data berhasil dimuat:
      mfcc_0     mfcc_1     mfcc_2     mfcc_3     mfcc_4     mfcc_5  \
0 -391.82520  86.733055  21.756060  13.757183   9.030744  12.091555   
1 -219.55458  95.143524  23.050543  11.273750  12.111441   7.253877   
2 -231.49965  95.491196  21.786690  16.874832   9.731382  10.766232   
3 -239.90352  93.402954  20.391285  14.796158   7.593359   8.971307   
4 -239.52228  96.243010  13.798207  11.316676   9.502037   7.641552   

      mfcc_6    mfcc_7    mfcc_8    mfcc_9  ...   mfcc_11   mfcc_12   mfcc_13  \
0  12.048173  0.365709  1.737657  5.475190  ... -6.118983 -5.571948 -4.757148   
1  12.820862 -2.045698 -0.279506  7.276806  ... -7.047171 -9.191520 -6.422854   
2  14.763392  2.315336  0.482845  3.260504  ... -1.712604 -7.362328 -1.683394   
3  12.700639 -0.081058  0.001442  4.289254  ... -2.979377 -8.099751 -2.410700   
4  16.703917  0.591387 -1.314283  5.297466  ... -1.369699 -7.202394 -7.726523   

    mfcc_14   mfcc_15   mfcc_16   mfcc_17   mfcc_18   mfcc_19  l